In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import pickle
import time
import cv2
import os
import tkinter
from tkinter import *
from tkinter import filedialog
from PIL import Image,ImageDraw,ImageTk


main = tkinter.Tk()
main.title("Pedestrian Detection")
main.geometry("1000x800")

l = ['Pedestrian','Not pedestrian']


def train_Model():

    model = Sequential()
   
    # Step 1 - Convolution
    model.add(Conv2D(32, (2, 2), input_shape = (160, 80, 3), activation = 'relu'))
   
    # Step 2 - Pooling
    model.add(MaxPooling2D(pool_size = (2, 2)))
   
    # Adding a second convolutional layer
    model.add(Conv2D(64, (2, 2), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
   
    # Adding a third convolutional layer
    model.add(Conv2D(128, (2, 2), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
   
    # Adding a fourth convolutional layer
    model.add(Conv2D(128, (2, 2), activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
   
    # Step 3 - Flattening
    model.add(Flatten())
   
    # Step 4 - Full connection
    model.add(Dense(units = 64, activation = 'relu'))
    model.add(Dense(units = 1, activation = 'sigmoid'))
   
    # Compiling the CNN
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    model.summary()

    batch_size = 128    
   
    train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2, zoom_range = 0.2,horizontal_flip = True)
                                       
    test_datagen = ImageDataGenerator(rescale = 1./255)
   
    training_set = train_datagen.flow_from_directory('INRIAPerson/Train',target_size = (160, 80), batch_size = 32,class_mode = 'binary')
   
    test_set = test_datagen.flow_from_directory('INRIAPerson/Test',target_size = (160, 80), batch_size = 32,class_mode = 'binary')

    hist = model.fit_generator(training_set,
                                steps_per_epoch = 5000//batch_size,
                                epochs = 10,
                                validation_data = test_set,
                                validation_steps = 1000//batch_size,
                                verbose=1)

    model.save("model/trained.h5")

    f = open('model/history.pckl', 'wb')
    pickle.dump(hist.history, f)
    f.close()

    f = open('model/history.pckl', 'rb')
    data = pickle.load(f)
    f.close()

    acc = data['accuracy']
    accuracy = acc[-1] * 100

    print("Accuracy:",accuracy,"\n",end="\n")

   
def detect():
    filename = filedialog.askopenfilename(initialdir="testImages")

    imagetest = image.load_img(filename, target_size = (160,80))

    im = Image.open(filename)
    image_arr = np.asarray(im)

    imagetest = image.img_to_array(imagetest)
    imagetest = np.expand_dims(imagetest,axis=0)

    # Initialize the HOG descriptor
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    detections, weights = hog.detectMultiScale(image_arr)

    # Convert to a normal Python list
   
    if detections==():
        print("\nNo pedestrian detected")
       
    else:
        detections_rectangles = detections.tolist()
        if os.path.exists('model/trained.h5'):
           
            detector = keras.models.load_model("model/trained.h5")

            #predict the output
            detector._make_predict_function()

        else:
            train_Model()
           
        predict = detector.predict(imagetest)

        print(detector.summary(),end='\n\n')
        print(l[np.argmax(predict)],end='\n\n')

        f = open('model/history.pckl', 'rb')
        data = pickle.load(f)
        f.close()

        acc = data['accuracy']
        accuracy = acc[-1] * 100

        print("Accuracy:",accuracy,"\n",end="\n")

        draw = ImageDraw.Draw(im)
        for x, y, w, h in detections_rectangles:
            draw.rectangle(
            [x, y, x + w, y + h], outline=(255, 0, 0))

        im.show()

def close():
    main.destroy()

font = ('Calibri', 16, 'bold')
title = Label(main, text='Pedestrian Detection',anchor=W, justify=CENTER)
title.config(bg='slateblue', fg='white')  
title.config(font=font)          
title.config(height=2, width=70)      
title.place(x=10,y=10)

#initialise text field
font1 = ('Calibri', 14)

#testing label
label = Label(main, text="Testing the model:")
label.config(font=font)
label.place(x=10,y=100)

#button to test a new input
testButton = Button(main, text="Upload Image",bg='firebrick1', fg='white', command=detect)
testButton.place(x=250,y=200)
testButton.config(font=font1)

#exit the user interface
exitButton = Button(main, text="Exit",bg='firebrick1', fg='white',command=close)
exitButton.place(x=300,y=500)
exitButton.config(font=font1)

main.config(bg='lightblue')
main.mainloop()

TclError: ignored